This notebook is used to calculate the simplified ductbank model in GridLAB-D based on a detailed model of a ductbank.

----

The ductbank model is the equilibrium thermal model (i.e., thermal mass is not considered) described by the following:

$$
    Q   \xrightarrow{} (T_C) 
        \xrightarrow{U_I} (T_S) 
        \xrightarrow{U_A} (T_I)
        \xrightarrow{U_D} (T_O)
        \xrightarrow{U_F} (T_G)
$$

where
- $Q$ is the thermal heat from the cable losses (input)
- $T_C$ is the temperature of the cable core
- $U_I$ is the thermal conductance of the cable insulation
- $T_S$ is the temperature of the cable shield
- $U_A$ is the thermal conductance of the duct air gap
- $T_I$ is the temperature of the duct inner surface
- $U_D$ is the thermal conductance of the duct core
- $T_O$ is the temperature of the duct outer surface
- $U_F$ is the thermal conductance of the fill soil
- $T_G$ is the temperature of the surface ground

![Example Box 3x2](box3x2.png "Example Box 3x2")

$$
    \left[ \begin{matrix}
        -U_{vo}-U_{ho}-U_{hi}-U_{vi} & U_{hi} & 0 & 0 & 0 & 0
    \\
        U_{vi} & 0 & 0 & 0 & 0 & 0
    \\
        0 & 0 & 0 & 0 & 0 & 0
    \\
        0 & 0 & 0 & 0 & 0 & 0
    \\
        0 & 0 & 0 & 0 & 0 & 0
    \\
        0 & 0 & 0 & 0 & 0 & 0    
    \end{matrix} \right]
    \left[ \begin{matrix}
        T_{11} \\ T_{12} \\ T_{21} \\ T_{22} \\ T_{31} \\ T_{32}
    \end{matrix} \right]
    =
    \left[ \begin{matrix}
        U_{vo}+U_{ho} & 1 & 0 & 0 & 0 & 0 & 0
    \\
        0 & 0 & 1 & 0 & 0 & 0 & 0
    \\
        0 & 0 & 0 & 1 & 0 & 0 & 0
    \\
        0 & 0 & 0 & 0 & 1 & 0 & 0
    \\
        0 & 0 & 0 & 0 & 0 & 1 & 0
    \\
        0 & 0 & 0 & 0 & 0 & 0 & 1
    \end{matrix} \right]
    \left[ \begin{matrix}
        T_G \\ Q_{11} \\ Q_{12} \\ Q_{21} \\ Q_{22} \\ Q_{31} \\ Q_{32}
    \end{matrix} \right]
$$

In [1]:
T_ambient = 2500 # W.m/K
fill_RHO = 6000 # W.m/K
soil_RHO = 12000 # W.m/K
bank_D = 36 * 0.0254 # m
bank_H = 22.5 * 0.0254 # m
bank_W = 18.5 * 0.0254 # m
duct_R = 2.5 * 0.0254 # m
h_spacing = 7 * 0.0254 # m
v_spacing = 5.5 * 0.0254 # m